testing S1 imagery

In [12]:
import pandas as pd
import ee
from geemap import geemap
ee.Initialize()

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD#description

# 1) Tom's flooding example
Quick test vis of Tom's Cyclone Idai case study. Questions arising:
* how important is orbital direction? possibly should be used to filter before compositing?
* does it make sense to make monthly median composites of VH & VV data?
* do we need to z-score before adding to model? scaling [-50,1]] will be very different to S2 wtc
* map vis. in RGB with only 2 bands...? 

In [2]:
# RGB composite difficult with 2 band data!
radarVis = {
  'min': -50,
  'max': 1,
  'bands': ['VV','VH','VH'], 
}

In [6]:
# Target date
targdate = '2019-03-24'
#targdate = '2019-05-01'

# Define area of interest (AOI)
aoi_cord_pairs = [[33.44259958627015, -18.49604760905407],
                  [33.44259958627015, -20.664424742367366],
                  [36.43088083627015, -20.664424742367366],
                  [36.43088083627015, -18.49604760905407]]
aoi = ee.Geometry.Polygon([aoi_cord_pairs])

In [64]:
# filter collection
# Modified version of Tom's approach
filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           ee.Filter.equals("instrumentMode", "IW"),
           #ee.Filter.bounds(aoi),   # Tom uses this, but can't make it work? ## AttributeError: type object 'Filter' has no attribute 'bounds'
           ee.Filter.geometry(aoi),  # solution from:  https://github.com/google/earthengine-api/issues/83
           ee.Filter.date(targdate, ee.Date(targdate).advance(1, 'month'))
           ]

s1 = s1.filter(filters)

In [47]:
# # alternate 2 step approach...
# s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
#      .filterDate(targdate, ee.Date(targdate).advance(1, 'month')) \
#      .filterBounds(aoi)

# filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
#            ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
#            ee.Filter.equals("instrumentMode", "IW")]
# s1 = s1.filter(filters)

In [50]:
# make median composites
s1_median = s1.select('VV', 'VH').median()

In [65]:
#visualise 
Map = geemap.Map(center=(-19.5,35), zoom=8)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
Map.add_basemap('SATELLITE')
Map.addLayer(s1_median, radarVis, 's1_median')
Map

Map(center=[-19.5, 35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

# 2) Polesia Test

In [28]:
fp_train_ext1 = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
roi_train = geemap.shp_to_ee(fp_train_ext1)
AOI = roi_train

START_DATE_LIST=['2019-01-01', '2019-02-01', 
                 '2019-03-01', '2019-04-01', 
                 '2019-05-01', '2019-06-01', 
                 '2019-07-01', '2019-08-01',
                 '2019-09-01', '2019-10-01',
                 '2019-11-01', '2019-12-01']

START_DATE_LIST=['2018-01-01', '2018-02-01', 
                 '2018-03-01', '2018-04-01', 
                 '2018-05-01', '2018-06-01', 
                 '2018-07-01', '2018-08-01',
                 '2018-09-01', '2018-10-01',
                 '2018-11-01', '2018-12-01']

START_DATE_LIST=[d.strftime('%Y-%m-%d') for d in pd.date_range(start='2017-01-01', end='2017-12-01', freq='MS')]


filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.geometry(AOI)]

def s1_mask_border_noise(img):
    """
    Sentinel-1 data on GEE sometimes suffers from 'border noise' prior to May 2018 at swath edges.
    This needs to be masked in individual images via mapping before compositing images.
    This func is derived from the example code provided on the 
    GEE ee.ImageCollection("COPERNICUS/S1_GRD") pages
    """
    edge = img.lt(-30.0)
    masked_img = img.mask().And(edge.Not())
    return img.updateMask(masked_img)

In [25]:
for i, START_DATE in enumerate(START_DATE_LIST):
    print(START_DATE)
    END_DATE = ee.Date(START_DATE).advance(1, 'month')
    
    #collection
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
         .filterDate(START_DATE, END_DATE)   
    s1 = s1.filter(filters)
    
    # composites
    s1_median = (s1.select('VV', 'VH')
                 .map(s1_mask_border_noise)
                  .median()
                  .clip(roi_train.geometry())
                  .rename(f'{START_DATE[0:7]}_VV', f'{START_DATE[0:7]}_VH'))
    
    # stack
    if i == 0:
        median_stack = s1_median
    else:
        median_stack = median_stack.addBands(s1_median)

2017-01-01
2017-02-01
2017-03-01
2017-04-01
2017-05-01
2017-06-01
2017-07-01
2017-08-01
2017-09-01
2017-10-01
2017-11-01
2017-12-01


In [4]:
median_stack.getInfo()

{'type': 'Image',
 'bands': [{'id': '2019-01_VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2019-01_VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2019-02_VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2019-02_VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2019-03_VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0,

examine composites

In [7]:
type(median_stack)

ee.image.Image

In [27]:
#month_list = [1,2,3,4,5,6]
#month_list = [11]

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
Map.add_basemap('SATELLITE')
for i, START_DATE in enumerate(START_DATE_LIST[0:12]):
    vis = {'min': -50,'max': 1,'bands': [f'{START_DATE[0:7]}_VV', 
                                         f'{START_DATE[0:7]}_VH', 
                                         f'{START_DATE[0:7]}_VH']}
    
    vis = {'min': -50,'max': 0,'bands': [f'{START_DATE[0:7]}_VH']}
    Map.addLayer(median_stack, vis, f'S1_{START_DATE}')
Map

# Jan-Feb2018 have a corrupt stipe of S1 data - not sure why, but these months need ignoring for now

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [105]:
#collection
END_DATE = ee.Date(START_DATE).advance(1, 'month')
print(START_DATE)

s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
     .filterDate(START_DATE, END_DATE)   

s1 = s1.filter(filters)

# composites
s1_median = s1.select('VV', 'VH').median()

#visualise 
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
Map.add_basemap('SATELLITE')

vis = {'min': -50,'max': 1,'bands': [f'{START_DATE[0:7]}_VV', 
                                     f'{START_DATE[0:7]}_VH', 
                                     f'{START_DATE[0:7]}_VH']}

vis = {'min': -50,'max': 1,'bands': [f'VV', 
                                     f'VH', 
                                     f'VH']}

Map.addLayer(s1_median, radarVis, 's1_median')
Map

2019-12-01


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

# 3) Test flooding frequency maps

In [6]:
fp_train_ext1 = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
roi_train = geemap.shp_to_ee(fp_train_ext1)
AOI = roi_train

START_DATE_LIST=['2019-01-01', '2019-02-01', 
                 '2019-03-01', '2019-04-01', 
                 '2019-05-01', '2019-06-01', 
                 '2019-07-01', '2019-08-01',
                 '2019-09-01', '2019-10-01',
                 '2019-11-01', '2019-12-01']

# START_DATE_LIST=['2018-01-01', '2018-02-01', 
#                  '2018-03-01', '2018-04-01', 
#                  '2018-05-01', '2018-06-01', 
#                  '2018-07-01', '2018-08-01',
#                  '2018-09-01', '2018-10-01',
#                  '2018-11-01', '2018-12-01']

filters = [ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           #ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.geometry(AOI)]

In [29]:
import pandas as pd
START_DATE_LIST=[d.strftime('%Y-%m-%d') for d in pd.date_range(start='2017-01-01', end='2019-12-31', freq='MS')]
SMOOTHING_RADIUS = 100
FLOOD_THRESH = -13

# Jan and Feb 2018 have a corrupt stipe of S1 data - not sure why, but these months need ignoring for now


for i, START_DATE in enumerate(START_DATE_LIST):
    print(START_DATE)
    END_DATE = ee.Date(START_DATE).advance(1, 'month')
    
    #collection
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
         .filterDate(START_DATE, END_DATE)   
    s1 = s1.filter(filters)
    
    # composites
    s1_median = (s1.select('VV'#, 
                           #'VH'
                          )
                  .map(s1_mask_border_noise)
                  .median()
                  .clip(roi_train.geometry())
                  .rename(f'{START_DATE[0:7]}_VV'#, f'{START_DATE[0:7]}_VH'
                         )
                )
    
    # smooth image to get rid of backscatter noise
    s1_smoothed = s1_median.focal_median(SMOOTHING_RADIUS, 'circle', 'meters')
    # apply a flood masking threshold
    flood = s1_smoothed.lt(FLOOD_THRESH)

    # stack
    if i == 0:
        median_stack = s1_median
        smooth_stack = s1_smoothed
        flood_stack = flood
        count_stack = flood.rename('VV')
        
    else:
        median_stack = median_stack.addBands(s1_median)
        smooth_stack = smooth_stack.addBands(s1_smoothed) 
        flood_stack = flood_stack.addBands(flood)
        count_stack = count_stack.add(flood.rename('VV'))

2017-01-01
2017-02-01
2017-03-01
2017-04-01
2017-05-01
2017-06-01
2017-07-01
2017-08-01
2017-09-01
2017-10-01
2017-11-01
2017-12-01
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01


In [30]:
#visualise 
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
Map.add_basemap('SATELLITE')

vis = {'min': -50,'max': 1,'bands': [f'2019-01_VV']}

Map.addLayer(median_stack, vis, 's1_median')
Map.addLayer(smooth_stack, vis, 's1_smooth')
vis = {'min': 0,'max': 1,'bands': [f'2019-01_VV']}
Map.addLayer(flood_stack, vis, 'flooded')
vis = {'min': 0,'max': 36,'bands': [f'VV']}
Map.addLayer(count_stack, vis, 'flooded_count')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [58]:
def fetch_sentinel1_flood_index_v1(aoi, start_date_str, end_date_str, smoothing_radius=100.0, flood_thresh=-13.0):
    """
    Create a simple flood frequency layer from Sentinel-1 data (IW mode & VV)
    1) preprocess median monthly composites using start/end dates
    2) apply spatial smoother (focal median) to get rid of the backscatter
    3) threshold monthly composite as binary flooded/not flooded map
    4) accumulate binary monthly flood maps, and normalise by dividing by number of months 
    
    NOTE: this func could be modified to make use of 'VH' polorisation.Needs further exploration. 
    This paper however suggests VV might be better for flood detection:
    https://iopscience.iop.org/article/10.1088/1755-1315/357/1/012034/pdf
    
    :: params ::
    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    : param start_date_str: str, should be YYYY-MM-01 for start month (inclusive) 
    : param end_date_str: str, should be YYYY-MM-01 for end month (inclusive)  
    : param smoothing_radius: float, radius of smoothing kernel (metres)
    : param flood_thresh: float, VV values < flood_thresh are considered flooded.
    : returns : flood frequency map [0-1]
    """

    print('fetch_sentinel1_flood_index_v1(): hello!')
    
    
    def s1_mask_border_noise(img):
        """
        Sentinel-1 data on GEE sometimes suffers from 'border noise' prior to May 2018 at swath edges.
        This needs to be masked in individual images via mapping before compositing images.
        This func is derived from the example code provided on the 
        GEE ee.ImageCollection("COPERNICUS/S1_GRD") pages
        """
        edge = img.lt(-30.0)
        masked_img = img.mask().And(edge.Not())
        return img.updateMask(masked_img)
    
    band='VV' # threshold needs changing if using VH and further testing 
    
    # generate list of months
    start_date_list=[d.strftime('%Y-%m-%d') for d in pd.date_range(start=start_date_str, 
                                                                   end=end_date_str, freq='MS')]
    n_months = len(start_date_list)  # used for standardising
    
    # specify filters to apply to the GEE Sentinel-1 collection
    filters = [ee.Filter.listContains("transmitterReceiverPolarisation", band),
           ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.geometry(aoi)]
    
    # iteratively generate a monthly flood map and aggregate them
    for i, start_date in enumerate(start_date_list):
        print(f'fetch_sentinel1_flood_index_v1(): processing month {start_date}')
        end_date = ee.Date(start_date).advance(1, 'month')
        
        # load, preprocess and make composite
        s1_median = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterDate(start_date, end_date)
             .filter(filters)
             .select(band)
             .map(s1_mask_border_noise) # remove border noise
             .median()
             .clip(aoi.geometry()))
        
        # smooth image to get rid of backscatter noise
        s1_smoothed = s1_median.focal_median(smoothing_radius, 'circle', 'meters')        
        
        # apply a flood masking threshold
        flood_map = s1_smoothed.lt(flood_thresh) 
        
        # sum months
        if i == 0:
            flood_aggr = flood_map
        else:
            flood_aggr = flood_aggr.add(flood_map)
    
    ## standardise & rename
    flood_aggr = flood_aggr.divide(n_months).rename("s1_floodfreq")
    
    print('fetch_sentinel1_flood_index_v1(): bye!')    
    return flood_aggr

s1 = fetch_sentinel1_flood_index_v1(aoi=roi_train, 
                                    start_date_str='2017-01-01', 
                                    end_date_str='2019-12-31'#, 
                                    #smoothing_radius=100.0, 
                                    #flood_thresh=-13.0
                                   )

fetch_sentinel1_flood_index_v1(): hello!
fetch_sentinel1_flood_index_v1(): processing month 2017-01-01
fetch_sentinel1_flood_index_v1(): processing month 2017-02-01
fetch_sentinel1_flood_index_v1(): processing month 2017-03-01
fetch_sentinel1_flood_index_v1(): processing month 2017-04-01
fetch_sentinel1_flood_index_v1(): processing month 2017-05-01
fetch_sentinel1_flood_index_v1(): processing month 2017-06-01
fetch_sentinel1_flood_index_v1(): processing month 2017-07-01
fetch_sentinel1_flood_index_v1(): processing month 2017-08-01
fetch_sentinel1_flood_index_v1(): processing month 2017-09-01
fetch_sentinel1_flood_index_v1(): processing month 2017-10-01
fetch_sentinel1_flood_index_v1(): processing month 2017-11-01
fetch_sentinel1_flood_index_v1(): processing month 2017-12-01
fetch_sentinel1_flood_index_v1(): processing month 2018-01-01
fetch_sentinel1_flood_index_v1(): processing month 2018-02-01
fetch_sentinel1_flood_index_v1(): processing month 2018-03-01
fetch_sentinel1_flood_index_v

In [50]:
s1.getInfo()

{'type': 'Image',
 'bands': [{'id': 's1_floodfreq',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [57]:
#visualise 
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
Map.add_basemap('SATELLITE')
vis = {'min': 0,'max': 1,'bands': [f's1_floodfreq']}
Map.addLayer(s1, vis, 'flooded_count')
Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…